## Implement a simple sentiment analysis model using word embeddings.

In [1]:
import nltk
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split

# Sample dataset (Positive and Negative Sentences)
data = [
    ("I love this product, it's amazing!", 1),
    ("This is the best movie I have ever seen", 1),
    ("Fantastic experience, I highly recommend it.", 1),
    ("Absolutely horrible, I hate it.", 0),
    ("Worst purchase ever, total waste of money.", 0),
    ("Not worth the price, very disappointing.", 0),
]

# Preprocess text
sentences, labels = zip(*data)
sentences = [nltk.word_tokenize(sent.lower()) for sent in sentences]

# Train Word2Vec model
word2vec = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Tokenize words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([" ".join(sent) for sent in sentences])
vocab_size = len(tokenizer.word_index) + 1

# Convert words to sequences
sequences = tokenizer.texts_to_sequences([" ".join(sent) for sent in sentences])
max_length = max(len(seq) for seq in sequences)
X = pad_sequences(sequences, maxlen=max_length, padding="post")
y = np.array(labels)

# Prepare embedding matrix
embedding_dim = 100
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in word2vec.wv:
        embedding_matrix[i] = word2vec.wv[word]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build Sentiment Analysis Model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False),
    LSTM(64, return_sequences=False),
    Dense(1, activation="sigmoid")
])

# Compile and Train Model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, batch_size=2, validation_data=(X_test, y_test))

# Test Prediction
test_sentence = "I really love this product"
test_seq = tokenizer.texts_to_sequences([test_sentence])
test_seq = pad_sequences(test_seq, maxlen=max_length, padding="post")
prediction = model.predict(test_seq)[0][0]

print(f"Sentiment Score: {prediction:.4f} ({'Positive' if prediction > 0.5 else 'Negative'})")



ImportError: cannot import name 'triu' from 'scipy.linalg' (C:\ProgramData\anaconda3\Lib\site-packages\scipy\linalg\__init__.py)